In [44]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import os

In [45]:
SL = pd.read_csv("C:/Users/josen/Documents/MASTER/TFM/SmartLifter.csv")

In [46]:
# Preprocesar los datos
# Seleccionar las columnas relevantes (consistente con la función de recomendación)
columns_to_use = ['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'Frequency']
SL[columns_to_use + ['Title']]

,Level_x,Age,Weight_kg,Height_m,Type,Frequency,Title
0,Intermediate,32,68.1,1.66,Cardio,4,Roc Everest
1,Intermediate,32,68.1,1.66,Cardio,4,HM Running Man Crunch
2,Intermediate,32,68.1,1.66,Cardio,4,Agility Ladder
3,Intermediate,32,68.1,1.66,Cardio,4,Swimming
4,Intermediate,32,68.1,1.66,Cardio,4,Rower
...,...,...,...,...,...,...,...
292387,Beginner,46,88.7,1.63,Strength,2,Sled Overhead Triceps Extension
292388,Beginner,46,88.7,1.63,Strength,2,Suspended Triceps Press
292389,Beginner,46,88.7,1.63,Strength,2,Low Cable Triceps Extension
292390,Beginner,46,88.7,1.63,Strength,2,Standing Towel Triceps Extension


In [47]:
# Codificar variables categóricas
label_encoders = {}
for col in ['Level_x', 'Type']:  # Solo codificar las categóricas usadas
    le = LabelEncoder()
    SL[col] = le.fit_transform(SL[col])
    label_encoders[col] = le

In [48]:
# Normalizar las variables continuas (incluyendo las codificadas)
scaler = StandardScaler()
SL[['Age', 'Weight_kg', 'Height_m', 'Frequency', 'Type', 'Level_x']] = scaler.fit_transform(SL[['Age', 'Weight_kg', 'Height_m', 'Frequency', 'Type', 'Level_x']])

In [49]:
# 3. Dividir los datos en entrenamiento y prueba
X = SL[['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'Frequency']]  # Incluir 'Level_x' y 'Type'
y = SL['Title']  # El título del ejercicio es la etiqueta

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [50]:
# 4. Entrenar el modelo Random Forest (sin muestreo, si es posible)

model = RandomForestClassifier(n_estimators=200, random_state=42)  # Aumentar estimadores si es posible
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPrecisión del modelo: {accuracy:.2f}")


Precisión del modelo: 0.00


In [51]:
# 5. Guardar el modelo y los preprocesadores
joblib.dump(model, 'random_forest_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("\nModelo y preprocesadores guardados exitosamente.")


Modelo y preprocesadores guardados exitosamente.


In [52]:
# 6. Función para recomendar ejercicios (corregida)
def recomendar_ejercicios(model, user_features, label_encoders, scaler, num_ejercicios=6, dias=7):
    """
    Recomienda ejercicios basados en las características del usuario.

    Args:
        model: Modelo Random Forest entrenado.
        user_features: Diccionario con las características del usuario (nivel, edad, peso, altura, frecuencia, tipo).  Asegúrate de que incluya 'Level_x' y 'Type'.
        label_encoders: Diccionario con los codificadores de las variables categóricas.
        scaler: Escalador para normalizar las características continuas.
        num_ejercicios: Número de ejercicios por día.
        dias: Número de días en la rutina.

    Returns:
        Una rutina semanal de ejercicios.
    """
    user_data = pd.DataFrame([user_features])

    # Codificar las columnas categóricas
    for col in ['Level_x', 'Type']: # Solo codificar Level_x y Type
        user_data[col] = label_encoders[col].transform(user_data[col])

    # Normalizar las columnas continuas
    user_data[['Age', 'Weight_kg', 'Height_m', 'Frequency', 'Type', 'Level_x']] = scaler.transform(user_data[['Age', 'Weight_kg', 'Height_m', 'Frequency', 'Type', 'Level_x']])

    # Ensure the order of columns matches the training data
    user_data = user_data[['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'Frequency']]

    # Predecir los ejercicios
    predicciones = model.predict_proba(user_data)
    top_indices = np.argsort(predicciones, axis=1)[:, -num_ejercicios * dias:][0]

    # Obtener los títulos de los ejercicios recomendados
    ejercicios_recomendados = y.iloc[top_indices].values

    # Asegurar que no se repitan ejercicios el mismo día ni al día siguiente
    rutina_semanal = []
    usados = set()
    for i in range(dias):
        dia_ejercicios = []
        for ejercicio in ejercicios_recomendados:
            if ejercicio not in usados and len(dia_ejercicios) < num_ejercicios:
                dia_ejercicios.append(ejercicio)
                usados.add(ejercicio)
        rutina_semanal.append(dia_ejercicios)

    return rutina_semanal

In [53]:
# 7. Ejemplo de uso (corregido, celda 147)
user_features = {
    'Level_x': 'Intermediate',  # Incluye 'Level_x'
    'Age': 25,
    'Weight_kg': 68,
    'Height_m': 1.75,
    'Type': 'Strength', # Incluye 'Type'
    'Frequency': 6
}

# Cargar el modelo y los preprocesadores
loaded_model = joblib.load('random_forest_model.pkl')
loaded_label_encoders = joblib.load('label_encoders.pkl')
loaded_scaler = joblib.load('scaler.pkl')


# Mostrar la rutina semanal
print("\nRutina semanal recomendada:")

# Use the loaded model, label encoders, and scaler to recommend exercises
rutina_semanal = recomendar_ejercicios(loaded_model, user_features, loaded_label_encoders, loaded_scaler)

for dia, ejercicios in enumerate(rutina_semanal, 1): # Corrige 'rutina' a 'rutina_semanal' y añade +1 al índice
    print(f"Día {dia}: {', '.join(ejercicios)}")


Rutina semanal recomendada:
Día 1: Crunch - Gethin Variation, FYR Sprawl Frog Kick, Elliptical trainer, FYR2 Kettlebell Swing-Clean-Press-Row Combo, FYR2 Alternating Kettlebell Snatch, HM In-And-Out Squat
Día 2: Push-Up Wide, 30 Arms Incline Biceps Curl, Close-Hands Push-Up, FYR2 Monster Band Face-Pull Hold, 30 Arms BFR Machine Preacher Curl, Dumbbell front squat
Día 3: Plate Row, Holman Plate Burpee, Holman Renegade Punch-Under, UN Barbell Rack Pull, Flexor Incline Dumbbell Curls, Holman Slow Deadlift
Día 4: Holman Supine Double Pike, 30 Hanging Toes-To-Bar, Partner sit-up with high-five, 30 Spider Crawl, Squatting cable EZ-bar biceps curl, Incline dumbbell chest fly
Día 5: Single-leg cable hip extension, Kettlebell One-Legged Deadlift, HM Air Jumping Rope, Incline Dumbbell Press - Gethin Variation, FYR2 Kettlebell Juggle, 30 Chest Low-To-High Cable Fly
Día 6: Dumbbell Squat Snatch, Holman Alternating Lunge Jump, Single-kettlebell windmill, Close-Grip EZ-Bar Curl with Band, MetaBurn 